In [1]:
import os, sys
import tensorflow as tf
import pandas as pd
import numpy as np
import dataclasses

2023-11-05 22:28:47.643813: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.12.0'

In [3]:
@dataclasses.dataclass
class params:
    model_pb_dir: str = "./lstm_model_2023-11-05-2/"

    def __init__(self, model_pb_dir: str):
        self.model_pb_dir = model_pb_dir
        self.model = tf.saved_model.load(model_pb_dir)

In [4]:
g_params = params( "./lstm_model_2023-11-05-2/" )

2023-11-05 22:28:55.759466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-11-05 22:28:55.759597: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-11-05 22:28:55.759680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [5]:
dir( g_params.model )

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_default_save_signature',
 '_deferred_dependencies',
 '_delete_tracking',
 '_deserialization_dependencies',
 '_deserialize_from_proto',
 '_export_to_saved_model_graph',
 '_gather_saveables_for_checkpoint',
 '_handle_deferred_dependencies',
 '_lookup_dependency',
 '_maybe_initialize_trackable',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_preload_simple_restoration',
 '_restore_from_tensors',
 '_self_name_based_restores',
 '_self_saveable_object

In [6]:
tf.config.run_functions_eagerly(True)
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [7]:
inference_func = g_params.model.signatures["serving_default"]

In [8]:
model_output = inference_func(tf.convert_to_tensor(np.ones((1,1,10), dtype = np.float32)))

In [9]:
model_output

{'dense': <tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.00891958]]], dtype=float32)>}

In [10]:
g_params.model.variables[0].shape, g_params.model.variables[1].shape, g_params.model.variables[2].shape, g_params.model.variables[3].shape, g_params.model.variables[4].shape, 

(TensorShape([10, 256]),
 TensorShape([64, 256]),
 TensorShape([256]),
 TensorShape([64, 1]),
 TensorShape([1]))

In [11]:
np.array(g_params.model.variables[0])

array([[ 0.00406891,  0.02942161, -0.07692266, ...,  0.0350378 ,
         0.14710282, -0.04384851],
       [-0.05274755,  0.14557846, -0.01052976, ...,  0.09109949,
         0.04527088, -0.01447561],
       [-0.07837641, -0.02560529,  0.07452903, ..., -0.04418456,
        -0.03712278,  0.07655569],
       ...,
       [ 0.12180015, -0.07375343, -0.10644257, ...,  0.09775096,
        -0.04994417,  0.00661113],
       [-0.04794744,  0.10409902, -0.02695975, ..., -0.12072746,
         0.09919485, -0.12267644],
       [ 0.16002688,  0.14137049,  0.11540732, ..., -0.08713548,
        -0.07780564, -0.06745956]], dtype=float32)

In [12]:
g_params.model._deserialize_from_proto()

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x161f13450>

In [13]:
class MLHelper:
    @classmethod
    def sigmoid(cls, arr):
        res = 1 / (1 + np.exp(-arr))
        return res

    @classmethod
    def tanh(cls, arr):
        return np.tanh(arr)

    @classmethod
    def relu( cls, arr):
        return np.maximum(0, x)

In [14]:
class LSTM:
    def __init__( self, X_Mat, H_Mat, B_Vec, init_hidden_state=None, init_cell_state=None ):
        self.X_Mat = X_Mat
        self.H_Mat = H_Mat
        self.B_Vec = B_Vec

        self.input_len = X_Mat.shape[0]
        self.unit = X_Mat.shape[1] // 4
        self.input_shape = (X_Mat.shape[0], 1)
        self.hidden_state_shape = (H_Mat.shape[0] , 1)
        self.cell_state_shape = self.hidden_state_shape
        self.timestep = 0

        if init_hidden_state is None:
            self.hidden_state = np.zeros(self.hidden_state_shape)
        if init_cell_state is None:
            self.cell_state = np.zeros(self.cell_state_shape)
        

        if X_Mat.shape[1] != H_Mat.shape[1]:
            raise ValueError(f"LSTM Input H_Mat Column Count ({H_Mat.shape[1]}) "\
                             f"Should Be The Same As X_Mat Column Count ({X_Mat.shape[1]})!")

        self.W_i, self.W_f, self.W_c, self.W_o = \
            X_Mat[:, :self.unit].T, X_Mat[:, self.unit: 2 * self.unit].T, X_Mat[:, 2 * self.unit: 3 * self.unit].T, X_Mat[:, 3 * self.unit:].T

        self.U_i, self.U_f, self.U_c, self.U_o = \
            H_Mat[:, :self.unit].T, H_Mat[:, self.unit: 2 * self.unit].T, H_Mat[:, 2 * self.unit: 3 * self.unit].T, H_Mat[:, 3 * self.unit:].T

        self.b_i, self.b_f, self.b_c, self.b_o = \
            B_Vec[:self.unit].reshape(self.hidden_state_shape),\
            B_Vec[self.unit: 2 * self.unit].reshape(self.hidden_state_shape),\
            B_Vec[2 * self.unit: 3 * self.unit].reshape(self.hidden_state_shape),\
            B_Vec[3 * self.unit:].reshape(self.hidden_state_shape)

    def infer_X( self, X_input ):
        a = np.matmul(self.W_o, X_input) + np.matmul(self.U_o, self.hidden_state) + self.b_o

        o_t = MLHelper.sigmoid(a)
        f_t = MLHelper.sigmoid(np.matmul(self.W_f, X_input) + np.matmul(self.U_f, self.hidden_state) + self.b_f)
        i_t = MLHelper.sigmoid(np.matmul(self.W_i, X_input) + np.matmul(self.U_i, self.hidden_state) + self.b_i)

        cell_state_ = MLHelper.tanh(np.matmul(self.W_c, X_input) + np.matmul(self.U_c, self.hidden_state) + self.b_c)

        self.cell_state = f_t * self.cell_state + i_t * cell_state_
        self.hidden_state = o_t * MLHelper.tanh(self.cell_state)
        self.timestep += 1
        return self

In [15]:
class Dense:
    def __init__(self, D_Mat, B_Vec):
        self.D_Mat = D_Mat
        self.B_Vec = B_Vec

        if self.D_Mat.shape[0] != self.B_Vec.shape[0]:
            raise ValueError(f"Dense Input D_Mat Column Count ({D_Mat.shape[1]}) "\
                             f"Should Be The Same As B_Vec Row Count ({B_Vec.shape[0]})!")

    def infer_X( self, X_input):
        self.out = np.matmul( self.D_Mat, X_input ) + self.B_Vec
        return self

In [16]:
trial_LSTM = LSTM( g_params.model.variables[0], g_params.model.variables[1], g_params.model.variables[2] )

In [17]:
input_X = np.ones((10, 1))

In [18]:
trial_LSTM.infer_X( input_X )

In [19]:
trial_LSTM.hidden_state.shape

(64, 1)

In [20]:
trial_Dense = Dense(np.array(g_params.model.variables[3]).T, np.array(g_params.model.variables[4]).reshape((1,1)))

In [21]:
trial_Dense.infer_X(trial_LSTM.hidden_state)

In [22]:
trial_Dense.out

array([[0.00891958]])